In [ ]:
#!conda activate tensorflow
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def info(data):
    print(data.columns)
    print(data.shape)
    registers = data.shape[0]
    print(registers)
    #print(data.count())

In [ ]:
def prepare_dates(data):
    data['FECHA_INGRESO'] = pd.to_datetime(data['FECHA_INGRESO'], format='%Y/%m/%d')
    data['FECHA_SINTOMAS'] = pd.to_datetime(data['FECHA_SINTOMAS'], format='%Y/%m/%d')

    data['STATUS'] = ["VIVO" if value=='9999-99-99' else "FALLECIDO" for value in data['FECHA_DEF']]

    data['FECHA_DEF'] = data['FECHA_DEF'].replace('9999-99-99',np.NaN)
    data['FECHA_DEF'] = pd.to_datetime(data['FECHA_DEF'], format='%Y/%m/%d')

    #data['FECHA_DEF'].replace('9999-99-99',np.NaN)
    #pd.to_datetime(data["FECHA_DEF"], format='%d/%m/%Y')
    #data.isnull().any()
    #data['FECHA_DEF'] = data['FECHA_DEF'] == '9999-99-99' ? '' : data['FECHA_DEF']

In [ ]:
#result = list(data[['ENTIDAD_RES','MUNICIPIO_RES']].to_records(index=False))

def replace_touple(df, col, dicc, colOut):
    salida = []
    salida = list()
    for key1, key2 in df[col]:
        if (key1,key2) in dicc:
            salida.append(dicc[(key1,key2)])
        else:
            salida.append(np.NaN)
       
    df[colOut] = salida

#tmp = [v in municipios for v in dataf['ENTIDAD_MUNICIPIO_RES']] 
#dataf.loc[tmp]

#dataf['ENTIDAD_MUNICIPIO_RES'].mask(dataf['ENTIDAD_MUNICIPIO_RES'] == (15,37), (1,1)), inplace=True)

In [ ]:
def apply_catalogues(data):
    xl_file = pd.ExcelFile("Reportes/Nuevos/diccionario_datos_covid19/Catalogos_0412.xlsx")
    dfs = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}

    origen = dfs['Catálogo ORIGEN'].set_index('CLAVE').to_dict()['DESCRIPCIÓN']
    sector = dfs['Catálogo SECTOR'].set_index('CLAVE').to_dict()['DESCRIPCIÓN']
    sexo = dfs['Catálogo SEXO'].set_index('CLAVE').to_dict()['DESCRIPCIÓN']
    tipo_paciente = dfs['Catálogo TIPO_PACIENTE'].set_index('CLAVE').to_dict()['DESCRIPCIÓN']
    si_no = dfs['Catálogo SI_NO'].set_index('CLAVE').to_dict()['DESCRIPCIÓN']
    nacionalidad = dfs['Catálogo NACIONALIDAD'].set_index('CLAVE').to_dict()['DESCRIPCIÓN']
    resultado = dfs['Catálogo RESULTADO'][1:].set_index('Unnamed: 0').to_dict()['Unnamed: 1']
    entidades = dfs['Catálogo de ENTIDADES'].set_index('CLAVE_ENTIDAD')['ABREVIATURA']
    municipios = dfs['Catálogo MUNICIPIOS'].set_index(['CLAVE_ENTIDAD','CLAVE_MUNICIPIO']).to_dict()['MUNICIPIO']
    
    dataf = data.copy()
    dataf['ENTIDAD_MUNICIPIO_RES'] = list(zip(dataf['ENTIDAD_RES'], dataf['MUNICIPIO_RES']))
    replace_touple(dataf, 'ENTIDAD_MUNICIPIO_RES', municipios, 'MUNICIPIO_RES')

    dataf = dataf.replace({'ORIGEN': origen})
    dataf = dataf.replace({'SECTOR': sector})
    dataf = dataf.replace({'ENTIDAD_UM': entidades})
    dataf = dataf.replace({'SEXO': sexo})
    dataf = dataf.replace({'ENTIDAD_NAC': entidades})
    dataf = dataf.replace({'ENTIDAD_RES': entidades})
    dataf = dataf.replace({'TIPO_PACIENTE': tipo_paciente})
    dataf = dataf.replace({'INTUBADO': si_no})
    dataf = dataf.replace({'NEUMONIA': si_no})
    dataf = dataf.replace({'NACIONALIDAD': nacionalidad})
    dataf = dataf.replace({'EMBARAZO': si_no})
    dataf = dataf.replace({'HABLA_LENGUA_INDI': si_no})
    dataf = dataf.replace({'DIABETES': si_no})
    dataf = dataf.replace({'EPOC': si_no})
    dataf = dataf.replace({'ASMA': si_no})
    dataf = dataf.replace({'INMUSUPR': si_no})
    dataf = dataf.replace({'HIPERTENSION': si_no})
    dataf = dataf.replace({'OTRA_CON': si_no})
    dataf = dataf.replace({'CARDIOVASCULAR': si_no})
    dataf = dataf.replace({'OBESIDAD': si_no})
    dataf = dataf.replace({'RENAL_CRONICA': si_no})
    dataf = dataf.replace({'TABAQUISMO': si_no})
    dataf = dataf.replace({'OTRO_CASO': si_no})
    dataf = dataf.replace({'RESULTADO': resultado})
    dataf = dataf.replace({'MIGRANTE': si_no})
    #dataf = dataf.replace({'PAIS_NACIONALIDAD': origen})
    #dataf = dataf.replace({'PAIS_ORIGEN': origen})
    dataf = dataf.replace({'UCI': si_no})

    dataf['DIAS_INGRESO_SINTOMAS'] = (dataf['FECHA_INGRESO'] - dataf['FECHA_SINTOMAS']).dt.days
    dataf['DIAS_SINTOMAS_DEF'] = (dataf['FECHA_DEF'] - dataf['FECHA_SINTOMAS']).dt.days
    
    return dataf

In [ ]:
data = pd.read_csv("Reportes/Nuevos/datos_abiertos_covid19/COVID19_Mexico.csv", sep=',', index_col=0)
data.head(7)

In [ ]:
prepare_dates(data)
info(data)

In [ ]:
dataf = apply_catalogues(data)
dataf.head(5)

In [ ]:
info(dataf)

In [ ]:
def plot_cases(data):
    #print(dataf.columns)
    dataq1 = data.query('RESULTADO =="Positivo SARS-CoV-2"')

    fechas_serie = pd.date_range('20200217', periods=78, freq='D')
    fechas_dataFrame = (fechas_serie.to_frame(name='Fecha'))

    fechaSintomas = dataq1.groupby(['FECHA_SINTOMAS']).count()['DIAS_INGRESO_SINTOMAS'].to_frame(name='FECHA_SINTOMAS')
    fechaIngreso = dataq1.groupby(['FECHA_INGRESO']).count()['DIAS_INGRESO_SINTOMAS'].to_frame(name='FECHA_INGRESO')
    fechadef = dataq1.groupby(['FECHA_DEF']).count()['DIAS_SINTOMAS_DEF'].to_frame(name='FECHA_DEF')

    fechas = fechas_dataFrame.join(fechaSintomas, lsuffix='si', rsuffix='_other')
    fechas = fechas.join(fechaIngreso, lsuffix='_caller', rsuffix='_other')
    fechas = fechas.join(fechadef, lsuffix='_caller', rsuffix='_other')

    fechas = fechas.drop(['Fecha'], axis=1)

    fechas.plot(kind='bar', figsize=(15,10))

In [ ]:
def prot_sick_times(data):
    #print(dataf.columns)
    dataq1 = data.query('RESULTADO =="Positivo SARS-CoV-2"')

    #sintomas = dataq1.groupby(['ENTIDAD_RES']).sum()['DIAS_SINTOMAS_DEF'].to_frame(name='FECHA_SINTOMAS')
    ingreso = dataq1.groupby(['ENTIDAD_RES'])['DIAS_INGRESO_SINTOMAS'].agg([np.mean]) #.agg([np.sum, pd.Series.count, np.mean, np.std])
    defunciones = dataq1.groupby(['ENTIDAD_RES'])['DIAS_SINTOMAS_DEF'].agg([np.mean])

    ingreso = ingreso.join(defunciones, lsuffix='_ingreso', rsuffix='_defunción')

    ingreso.plot(kind='bar', figsize=(15,10))

In [ ]:
plot_cases(dataf)
prot_sick_times(dataf)

In [ ]:
#print(dataf.columns)
#dataq1 = dataf.query('RESULTADO =="Positivo SARS-CoV-2" and ENTIDAD_UM == "DF"')
dataq1 = dataf.query('RESULTADO =="Positivo SARS-CoV-2"')

sintomas = dataq1.groupby(['ENTIDAD_RES'])['FECHA_INGRESO'].agg(['count'])
ingreso = dataq1.groupby(['ENTIDAD_RES'])['FECHA_SINTOMAS'].agg(['count']) #.agg([np.sum, pd.Series.count, np.mean, np.std])
defunciones = dataq1.groupby(['ENTIDAD_RES'])['FECHA_DEF'].agg(['count'])

#sintomas = sintomas.join(ingreso, lsuffix='_ingreso', rsuffix='_sintomas')
ingreso = ingreso.join(defunciones, lsuffix='_ingreso', rsuffix='_defunciones')

ingreso.plot(kind='bar', figsize=(15,10))

In [ ]:
#print(dataf.columns)
#dataq1 = dataf.query('RESULTADO =="Positivo SARS-CoV-2" and ENTIDAD_UM == "JC" and STATUS == "FALLECIDO"')
dataq1 = dataf.query('RESULTADO =="Positivo SARS-CoV-2" and STATUS == "FALLECIDO"')
#dataq1 = dataf.query('RESULTADO =="Positivo SARS-CoV-2"')
dataq1 = dataq1[['SECTOR','ENTIDAD_UM','TIPO_PACIENTE','FECHA_INGRESO','FECHA_SINTOMAS','FECHA_DEF','STATUS']]

#dataq1 = dataq1.groupby(['ENTIDAD_RES','SECTOR'])['DIAS_INGRESO_SINTOMAS'].agg(['count']).unstack(1)

dataq1 = dataq1.groupby(['SECTOR','TIPO_PACIENTE','STATUS'])['ENTIDAD_UM'].agg(['count']).unstack(1)


print(dataq1)

dataq1.plot(kind='bar', figsize=(15,10))

In [ ]:
estado = "JC"
#print(dataf.columns)
#dataq1 = dataf.query('RESULTADO =="Positivo SARS-CoV-2"')
#dataq1 = dataq1[['SECTOR','ENTIDAD_UM','TIPO_PACIENTE','FECHA_INGRESO','FECHA_SINTOMAS','FECHA_DEF','STATUS']]

#dataq1 = dataq1.groupby(['ENTIDAD_RES','SECTOR'])['DIAS_INGRESO_SINTOMAS'].agg(['count']).unstack(1)

#dataq1 = dataq1.groupby(['FECHA_INGRESO'])['ENTIDAD_UM'].agg(['count']).unstack(1)

#print(dataq1.head())

#dataq1.plot(kind='bar', figsize=(15,10))

dataq1 = dataf.query('RESULTADO =="Positivo SARS-CoV-2" and ENTIDAD_RES == "' + estado + '"')

fechaSintomas = dataq1.groupby(['FECHA_SINTOMAS']).count()['DIAS_INGRESO_SINTOMAS'].to_frame(name='FECHA_SINTOMAS')
fechaIngreso = dataq1.groupby(['FECHA_INGRESO']).count()['DIAS_INGRESO_SINTOMAS'].to_frame(name='FECHA_INGRESO')
fechadef = dataq1.groupby(['FECHA_DEF']).count()['DIAS_SINTOMAS_DEF'].to_frame(name='FECHA_DEF')

#date_time_str = '20200217'
#date_time_obj = datetime.datetime.strptime(date_time_str, '%Y%m%d')

dias = (fechaIngreso.iloc[-1,:].name-fechaIngreso.iloc[0,:].name).days + 1

fechas_serie = pd.date_range(fechaIngreso.iloc[0,:].name, periods=dias, freq='D')
fechas_dataFrame = (fechas_serie.to_frame(name='Fecha'))

fechas = fechas_dataFrame.join(fechaIngreso, lsuffix='si', rsuffix='_other')
#fechas = fechas.join(fechaIngreso, lsuffix='_caller', rsuffix='_other')
#fechas = fechas.join(fechadef, lsuffix='_caller', rsuffix='_other')

#fechas = fechas.drop(['Fecha'], axis=1)
fechas = fechas.set_index('Fecha')

#%timeit fechas['ma'] = fechas['FECHA_INGRESO'].rolling(5).mean()
fechas['MA'] = fechas['FECHA_INGRESO'].rolling(5).mean()



#fechas.plot(kind='bar', figsize=(15,10))

#fig = plt.figure()
ax = fechas.plot(kind='bar',grid=False, figsize=(15,10), y='FECHA_INGRESO')
ax = ax.twinx()
#fechas.plot(linestyle='-', marker='o', x='MA', ax=ax)

#ax2.plot(fechas, linestyle='-', linewidth=2.0, color='red', y='MA')
#plt.show();

#ax = df_13_14_target[['month','2014_target_val']].plot(x='month',linestyle='-', marker='o')
#df_13_14_target[['month','2013_val','2014_val']].plot(x='month', kind='bar', ax=ax)


#print(dataf.columns)def graficosPorResultado(estado, consulta = 'RESULTADO in("Positivo SARS-CoV-2","Resultado pendiente")', grafica = 'bar'):
    textoEstado = 'and ENTIDAD_RES == "' + estado + '"' if estado != '' else ''
    textoConsulta = consulta if consulta != '' else 'RESULTADO in("Positivo SARS-CoV-2","Resultado pendiente")'
    grafica = grafica if grafica != '' else 'bar'
    
    dataq1 = dataf.query(textoConsulta + ' ' + textoEstado)
    #dataq1 = dataf.query('ENTIDAD_RES == "' + estado + '" and TIPO_PACIENTE == "HOSPITALIZADO"')

    fechaSintomas = dataq1.groupby(['FECHA_SINTOMAS','RESULTADO']).count()['DIAS_INGRESO_SINTOMAS'].to_frame(name='SINTOMAS')
    fechaIngreso = dataq1.groupby(['FECHA_INGRESO','RESULTADO']).count()['DIAS_INGRESO_SINTOMAS'].to_frame(name='INGRESOS')
    fechadef = dataq1.groupby(['FECHA_DEF','RESULTADO']).count()['DIAS_SINTOMAS_DEF'].to_frame(name='DEF')

    fechaIngreso = pd.pivot_table(fechaIngreso, values='INGRESOS', index=['FECHA_INGRESO'], columns=['RESULTADO'], aggfunc=np.sum)

    dias = (fechaIngreso.iloc[-1,:].name-fechaIngreso.iloc[0,:].name).days + 1
    fechas_serie = pd.date_range(fechaIngreso.iloc[0,:].name, periods=dias, freq='D')
    fechas_dataFrame = (fechas_serie.to_frame(name='Fecha'))

    fechas = fechas_dataFrame.join(fechaIngreso, lsuffix='si', rsuffix='_other')
    #fechas = fechas.join(fechaIngreso, lsuffix='_caller', rsuffix='_other')
    #fechas = fechas.join(fechadef, lsuffix='_caller', rsuffix='_other')

    fechas = fechas.set_index('Fecha')

    #%timeit fechas['ma'] = fechas['FECHA_INGRESO'].rolling(5).mean()
    #fechas['MA'] = fechas['Positivo SARS-CoV-2'].rolling(5).mean()

    fechas.plot(kind=grafica,stacked=True,title="Ingresos", figsize=(15,10))


    fechaSintomas = pd.pivot_table(fechaSintomas, values='SINTOMAS', index=['FECHA_SINTOMAS'], columns=['RESULTADO'], aggfunc=np.sum)

    dias = (fechaSintomas.iloc[-1,:].name-fechaSintomas.iloc[0,:].name).days + 1
    fechas_serie = pd.date_range(fechaSintomas.iloc[0,:].name, periods=dias, freq='D')
    fechas_dataFrame = (fechas_serie.to_frame(name='Fecha'))

    fechas = fechas_dataFrame.join(fechaSintomas, lsuffix='si', rsuffix='_other')

    fechas = fechas.set_index('Fecha')

    #%timeit fechas['ma'] = fechas['FECHA_INGRESO'].rolling(5).mean()
    #fechas['MA'] = fechas['Positivo SARS-CoV-2'].rolling(5).mean()

    fechas.plot(kind=grafica,stacked=True,title="Sintomas", figsize=(15,10))

    fechadef = pd.pivot_table(fechadef, values='DEF', index=['FECHA_DEF'], columns=['RESULTADO'], aggfunc=np.sum)

    dias = (fechadef.iloc[-1,:].name-fechadef.iloc[0,:].name).days + 1
    fechas_serie = pd.date_range(fechadef.iloc[0,:].name, periods=dias, freq='D')
    fechas_dataFrame = (fechas_serie.to_frame(name='Fecha'))

    fechas = fechas_dataFrame.join(fechadef, lsuffix='si', rsuffix='_other')

    fechas = fechas.set_index('Fecha')

    #%timeit fechas['ma'] = fechas['FECHA_INGRESO'].rolling(5).mean()
    #fechas['MA'] = fechas['Positivo SARS-CoV-2'].rolling(5).mean()

    fechas.plot(kind=grafica,stacked=True,title="Defunciones", figsize=(15,10))

#display(fechas)
graficosPorResultado('JC')
#graficosPorResultado('JC', 'TIPO_PACIENTE == "AMBULATORIO"')
#graficosPorResultado('JC', 'RESULTADO in("Positivo SARS-CoV-2","Resultado pendiente","No positivo SARS-CoV-2")')
#graficosPorResultado('JC', 'RESULTADO in("Positivo SARS-CoV-2","No positivo SARS-CoV-2")', 'line')
#AMBULATORIO
#HOSPITALIZADO
#NO ESPECIFICADO
#dataq1 = dataf.query('RESULTADO =="Positivo SARS-CoV-2" and ENTIDAD_UM == "JC"')
dataq1 = dataf.query('RESULTADO =="Positivo SARS-CoV-2" and ENTIDAD_RES == "' + estado + '"')

fechaSintomas = dataq1.groupby(['FECHA_SINTOMAS']).count()['DIAS_INGRESO_SINTOMAS'].to_frame(name='FECHA_SINTOMAS')
fechaIngreso = dataq1.groupby(['FECHA_INGRESO']).count()['DIAS_INGRESO_SINTOMAS'].to_frame(name='FECHA_INGRESO')
fechadef = dataq1.groupby(['FECHA_DEF']).count()['DIAS_SINTOMAS_DEF'].to_frame(name='FECHA_DEF')

#date_time_str = '20200217'
#date_time_obj = datetime.datetime.strptime(date_time_str, '%Y%m%d')

dias = (fechaSintomas.iloc[-1,:].name-fechaSintomas.iloc[0,:].name).days + 1

fechas_serie = pd.date_range(fechaSintomas.iloc[0,:].name, periods=dias, freq='D')
fechas_dataFrame = (fechas_serie.to_frame(name='Fecha'))


fechas = fechas_dataFrame.join(fechaSintomas, lsuffix='si', rsuffix='_other')
#fechas = fechas.join(fechaIngreso, lsuffix='_caller', rsuffix='_other')
#fechas = fechas.join(fechadef, lsuffix='_caller', rsuffix='_other')

#fechas = fechas.drop(['Fecha'], axis=1)
fechas = fechas.set_index('Fecha')

fechas.plot(kind='bar', figsize=(15,10))


#print(dataf.columns)
dataq1 = dataf.query('RESULTADO =="Positivo SARS-CoV-2" and ENTIDAD_RES == "' + estado + '"')

fechaSintomas = dataq1.groupby(['FECHA_SINTOMAS']).count()['DIAS_INGRESO_SINTOMAS'].to_frame(name='FECHA_SINTOMAS')
fechaIngreso = dataq1.groupby(['FECHA_INGRESO']).count()['DIAS_INGRESO_SINTOMAS'].to_frame(name='FECHA_INGRESO')
fechadef = dataq1.groupby(['FECHA_DEF']).count()['DIAS_SINTOMAS_DEF'].to_frame(name='FECHA_DEF')

#date_time_str = '20200217'
#date_time_obj = datetime.datetime.strptime(date_time_str, '%Y%m%d')

dias = (fechadef.iloc[-1,:].name-fechadef.iloc[0,:].name).days + 1

fechas_serie = pd.date_range(fechadef.iloc[0,:].name, periods=dias, freq='D')
fechas_dataFrame = (fechas_serie.to_frame(name='Fecha'))


fechas = fechas_dataFrame.join(fechadef, lsuffix='si', rsuffix='_other')
#fechas = fechas.join(fechaIngreso, lsuffix='_caller', rsuffix='_other')
#fechas = fechas.join(fechadef, lsuffix='_caller', rsuffix='_other')

#fechas = fechas.drop(['Fecha'], axis=1)
fechas = fechas.set_index('Fecha')

fechas.plot(kind='bar', figsize=(15,10))

In [ ]:
#ingreso = ingreso.join(defunciones, lsuffix='_ingreso', rsuffix='_defunción')
#ingreso.plot(kind='bar', figsize=(15,10))

#ingreso.plot(kind='bar', figsize=(15,10))
#defunciones.plot(kind='bar', figsize=(15,10))

#fechas = fechas_dataFrame.join(fechaSintomas, lsuffix='si', rsuffix='_other')
#fechas = fechas.join(fechaIngreso, lsuffix='_caller', rsuffix='_other')
#fechas = fechas.join(fechadef, lsuffix='_caller', rsuffix='_other')

#fechas = fechas.drop(['Fecha'], axis=1)

#fechas.plot(kind='line', figsize=(15,10))

#print(dataq1[['ENTIDAD_RES','MUNICIPIO_RES','FECHA_SINTOMAS','FECHA_INGRESO','FECHA_DEF']])
#dataq1 = dataq1.groupby(['FECHA_SINTOMAS','ENTIDAD_RES'])['SEXO'].agg(['count']).unstack(1) 
#dataq1 = dataq1.groupby(['FECHA_SINTOMAS','ENTIDAD_RES'])['SEXO'].agg(['count']).unstack(1) 


#dataq1 = dataq1[['FECHA_INGRESO','FECHA_SINTOMAS','FECHA_DEF', 'DIAS_INGRESO_SINTOMAS','DIAS_SINTOMAS_DEF']]
#dataq1.head()

#print(dataq1.head(1))
#dataq1.plot(kind='line', figsize=(15,20))

#dataq1[['DIAS_INGRESO_SINTOMAS','DIAS_SINTOMAS_DEF']]
#dataq1 = dataq1.groupby(['FECHA_SINTOMAS','ENTIDAD_RES'])['SEXO'].agg(['count']).unstack(1)


#print(fechaSintomas)
#print(type(fechas))

#fechasIndex = pd.DatetimeIndex(fechas.values)
#df3=df.set_index(fechasIndex)
#print(type(df3))

#records = df.to_records(index=False)
#result = list(records)
#print(result)
#OUTPUT

dataq1 = dataf.query('RESULTADO =="Positivo SARS-CoV-2" and ENTIDAD_RES == "' + estado + '"')

fechas_serie = pd.date_range('20200217', periods=90, freq='D')
fechas_dataFrame = (fechas_serie.to_frame(name='Fecha'))

fechaSintomas = dataq1.groupby(['FECHA_SINTOMAS']).count()['DIAS_INGRESO_SINTOMAS'].to_frame(name='FECHA_SINTOMAS')
fechaIngreso = dataq1.groupby(['FECHA_INGRESO']).count()['DIAS_INGRESO_SINTOMAS'].to_frame(name='FECHA_INGRESO')
fechadef = dataq1.groupby(['FECHA_DEF']).count()['DIAS_SINTOMAS_DEF'].to_frame(name='FECHA_DEF')

fechas = fechas_dataFrame.join(fechaSintomas, lsuffix='si', rsuffix='_other')
fechas = fechas.join(fechaIngreso, lsuffix='_caller', rsuffix='_other')
fechas = fechas.join(fechadef, lsuffix='_caller', rsuffix='_other')

fechas = fechas.sum()

fechas

#fechas = fechas.drop(['Fecha'], axis=1)

#fechas.plot(kind='bar', figsize=(15,10))

In [ ]:
#print(dataf.columns)
dataq1 = dataf.query('RESULTADO =="Positivo SARS-CoV-2" and STATUS == "FALLECIDO"')

#sintomas = dataq1.groupby(['ENTIDAD_RES']).sum()['DIAS_SINTOMAS_DEF'].to_frame(name='FECHA_SINTOMAS')
ingreso = dataq1.groupby(['SECTOR'])['DIAS_INGRESO_SINTOMAS'].agg([np.mean]) #.agg([np.sum, pd.Series.count, np.mean, np.std])
defunciones = dataq1.groupby(['SECTOR'])['DIAS_SINTOMAS_DEF'].agg([np.mean])

ingreso = ingreso.join(defunciones, lsuffix='_ingreso', rsuffix='_defunción')

ingreso.plot(kind='bar', figsize=(15,10))

In [ ]:
def graficosPorResultado(estado, consulta = 'RESULTADO in("Positivo SARS-CoV-2","Resultado pendiente")', grafica = 'bar'):
    textoEstado = 'and ENTIDAD_RES == "' + estado + '"' if estado != '' else ''
    textoConsulta = consulta if consulta != '' else 'RESULTADO in("Positivo SARS-CoV-2","Resultado pendiente")'
    grafica = grafica if grafica != '' else 'bar'
    
    dataq1 = dataf.query(textoConsulta + ' ' + textoEstado)
    #dataq1 = dataf.query('ENTIDAD_RES == "' + estado + '" and TIPO_PACIENTE == "HOSPITALIZADO"')

    fechaSintomas = dataq1.groupby(['FECHA_SINTOMAS','RESULTADO']).count()['DIAS_INGRESO_SINTOMAS'].to_frame(name='SINTOMAS')
    fechaIngreso = dataq1.groupby(['FECHA_INGRESO','RESULTADO']).count()['DIAS_INGRESO_SINTOMAS'].to_frame(name='INGRESOS')
    fechadef = dataq1.groupby(['FECHA_DEF','RESULTADO']).count()['DIAS_SINTOMAS_DEF'].to_frame(name='DEF')

    fechaIngreso = pd.pivot_table(fechaIngreso, values='INGRESOS', index=['FECHA_INGRESO'], columns=['RESULTADO'], aggfunc=np.sum)

    dias = (fechaIngreso.iloc[-1,:].name-fechaIngreso.iloc[0,:].name).days + 1
    fechas_serie = pd.date_range(fechaIngreso.iloc[0,:].name, periods=dias, freq='D')
    fechas_dataFrame = (fechas_serie.to_frame(name='Fecha'))

    fechas = fechas_dataFrame.join(fechaIngreso, lsuffix='si', rsuffix='_other')
    #fechas = fechas.join(fechaIngreso, lsuffix='_caller', rsuffix='_other')
    #fechas = fechas.join(fechadef, lsuffix='_caller', rsuffix='_other')

    fechas = fechas.set_index('Fecha')

    #%timeit fechas['ma'] = fechas['FECHA_INGRESO'].rolling(5).mean()
    #fechas['MA'] = fechas['Positivo SARS-CoV-2'].rolling(5).mean()

    fechas.plot(kind=grafica,stacked=True,title="Ingresos", figsize=(15,10))


    fechaSintomas = pd.pivot_table(fechaSintomas, values='SINTOMAS', index=['FECHA_SINTOMAS'], columns=['RESULTADO'], aggfunc=np.sum)

    dias = (fechaSintomas.iloc[-1,:].name-fechaSintomas.iloc[0,:].name).days + 1
    fechas_serie = pd.date_range(fechaSintomas.iloc[0,:].name, periods=dias, freq='D')
    fechas_dataFrame = (fechas_serie.to_frame(name='Fecha'))

    fechas = fechas_dataFrame.join(fechaSintomas, lsuffix='si', rsuffix='_other')

    fechas = fechas.set_index('Fecha')

    #%timeit fechas['ma'] = fechas['FECHA_INGRESO'].rolling(5).mean()
    #fechas['MA'] = fechas['Positivo SARS-CoV-2'].rolling(5).mean()

    fechas.plot(kind=grafica,stacked=True,title="Sintomas", figsize=(15,10))

    fechadef = pd.pivot_table(fechadef, values='DEF', index=['FECHA_DEF'], columns=['RESULTADO'], aggfunc=np.sum)

    dias = (fechadef.iloc[-1,:].name-fechadef.iloc[0,:].name).days + 1
    fechas_serie = pd.date_range(fechadef.iloc[0,:].name, periods=dias, freq='D')
    fechas_dataFrame = (fechas_serie.to_frame(name='Fecha'))

    fechas = fechas_dataFrame.join(fechadef, lsuffix='si', rsuffix='_other')

    fechas = fechas.set_index('Fecha')

    #%timeit fechas['ma'] = fechas['FECHA_INGRESO'].rolling(5).mean()
    #fechas['MA'] = fechas['Positivo SARS-CoV-2'].rolling(5).mean()

    fechas.plot(kind=grafica,stacked=True,title="Defunciones", figsize=(15,10))

#display(fechas)
graficosPorResultado('JC')
#graficosPorResultado('JC', 'TIPO_PACIENTE == "AMBULATORIO"')
#graficosPorResultado('JC', 'RESULTADO in("Positivo SARS-CoV-2","Resultado pendiente","No positivo SARS-CoV-2")')
#graficosPorResultado('JC', 'RESULTADO in("Positivo SARS-CoV-2","No positivo SARS-CoV-2")', 'line')
#AMBULATORIO
#HOSPITALIZADO
#NO ESPECIFICADO

In [ ]:
estado = "JC"

dataq1 = dataf.query('RESULTADO =="Positivo SARS-CoV-2" and ENTIDAD_RES == "' + estado + '"')
medias_moviles = 7
diasAjuste = -0

fechaSintomas = dataq1.groupby(['FECHA_SINTOMAS']).count()['DIAS_INGRESO_SINTOMAS'].to_frame(name='FECHA_SINTOMAS')
fechaIngreso = dataq1.groupby(['FECHA_INGRESO']).count()['DIAS_INGRESO_SINTOMAS'].to_frame(name='FECHA_INGRESO')
fechadef = dataq1.groupby(['FECHA_DEF']).count()['DIAS_SINTOMAS_DEF'].to_frame(name='FECHA_DEF')

dias = (fechaIngreso.iloc[-1,:].name-fechaIngreso.iloc[0,:].name).days + 1 + diasAjuste
fechas_serie = pd.date_range(fechaIngreso.iloc[0,:].name, periods=dias, freq='D')
fechas_dataFrame = (fechas_serie.to_frame(name='Fecha'))
fechas = fechas_dataFrame.join(fechaIngreso, lsuffix='si', rsuffix='_other')
fechas = fechas.set_index('Fecha')

fechas['MA'] = fechas['FECHA_INGRESO'].rolling(medias_moviles).mean()
fechas['MA'] = fechas['MA'].replace(np.NaN,0)

fechas_ingreso = fechas.FECHA_INGRESO.to_frame(name='FECHA_INGRESO')
fechas_ma = fechas.MA.to_frame(name='MA')

#with pd.option_context('display.max_rows', None, 'display.max_columns', None): display(fechas)

fig = plt.figure()
ax = fechas_ingreso['FECHA_INGRESO'].plot(kind='bar',grid=False, figsize=(15,10))
ax2 = ax.twinx()
ax2.plot(fechas_ma['MA'].values, linestyle='-', linewidth=2.0,color='red')
plt.show();


dias = (fechaSintomas.iloc[-1,:].name-fechaSintomas.iloc[0,:].name).days + 1 + diasAjuste
fechas_serie = pd.date_range(fechaSintomas.iloc[0,:].name, periods=dias, freq='D')
fechas_dataFrame = (fechas_serie.to_frame(name='Fecha'))
fechas = fechas_dataFrame.join(fechaSintomas, lsuffix='si', rsuffix='_other')
fechas = fechas.set_index('Fecha')

fechas['MA'] = fechas['FECHA_SINTOMAS'].rolling(medias_moviles).mean()
fechas['MA'] = fechas['MA'].replace(np.NaN,0)

fechas_sintomas = fechas.FECHA_SINTOMAS.to_frame(name='FECHA_SINTOMAS')
fechas_ma = fechas.MA.to_frame(name='MA')

fig = plt.figure()
ax = fechas_sintomas['FECHA_SINTOMAS'].plot(kind='bar',grid=False, figsize=(15,10))
ax2 = ax.twinx()
ax2.plot(fechas_ma['MA'].values, linestyle='-', linewidth=2.0,color='red')
plt.show();


dias = (fechadef.iloc[-1,:].name-fechadef.iloc[0,:].name).days + 1 + diasAjuste
fechas_serie = pd.date_range(fechadef.iloc[0,:].name, periods=dias, freq='D')
fechas_dataFrame = (fechas_serie.to_frame(name='Fecha'))
fechas = fechas_dataFrame.join(fechadef, lsuffix='si', rsuffix='_other')
fechas = fechas.set_index('Fecha')

fechas['MA'] = fechas['FECHA_DEF'].rolling(medias_moviles).mean()
fechas['MA'] = fechas['MA'].replace(np.NaN,0)

fechas_defunciones = fechas.FECHA_DEF.to_frame(name='FECHA_DEF')
fechas_ma = fechas.MA.to_frame(name='MA')

fig = plt.figure()
ax = fechas_defunciones['FECHA_DEF'].plot(kind='bar',grid=False, figsize=(15,10))
ax2 = ax.twinx()
ax2.plot(fechas_ma['MA'].values, linestyle='-', linewidth=2.0,color='red')
plt.show();

In [ ]:
def graficosPorResultadoMA(estado, consulta = 'RESULTADO in("Positivo SARS-CoV-2","Resultado pendiente")', grafica = 'bar'):
    
    textoEstado = 'and ENTIDAD_RES == "' + estado + '"' if estado != '' else ''
    textoConsulta = consulta if consulta != '' else 'RESULTADO in("Positivo SARS-CoV-2","Resultado pendiente")'
    grafica = grafica if grafica != '' else 'bar'
    
    dataq1 = dataf.query(textoConsulta + ' ' + textoEstado)
    
    fechaSintomas = dataq1.groupby(['FECHA_SINTOMAS','RESULTADO']).count()['DIAS_INGRESO_SINTOMAS'].to_frame(name='SINTOMAS')
    fechaIngreso = dataq1.groupby(['FECHA_INGRESO','RESULTADO']).count()['DIAS_INGRESO_SINTOMAS'].to_frame(name='INGRESOS')
    fechadef = dataq1.groupby(['FECHA_DEF','RESULTADO']).count()['DIAS_SINTOMAS_DEF'].to_frame(name='DEF')
    
    medias_moviles = 7
    diasAjuste = -0

    fechaIngreso = pd.pivot_table(fechaIngreso, values='INGRESOS', index=['FECHA_INGRESO'], columns=['RESULTADO'], aggfunc=np.sum)

    dias = (fechaIngreso.iloc[-1,:].name-fechaIngreso.iloc[0,:].name).days + 1 + diasAjuste
    fechas_serie = pd.date_range(fechaIngreso.iloc[0,:].name, periods=dias, freq='D')
    fechas_dataFrame = (fechas_serie.to_frame(name='Fecha'))

    fechas = fechas_dataFrame.join(fechaIngreso, lsuffix='si', rsuffix='_other')

    fechas = fechas.set_index('Fecha')

    %timeit fechas['MA'] = fechas.iloc[:,0].rolling(medias_moviles).mean()
    fechas['MA'] = fechas['MA'].replace(np.NaN,0)
    
    #fechas[fechas.columns[0:len(fechas.columns) - 1]].plot(kind=grafica,stacked=True,title="Ingresos", figsize=(15,10))
    
    fig = plt.figure()
    ax = fechas[fechas.columns[0:len(fechas.columns) - 1]].plot(kind='bar',title="Ingresos",stacked=True,grid=False, figsize=(15,10))
    ax2 = ax.twinx()
    ax2.plot(fechas[fechas.columns[len(fechas.columns) - 1]].values, linestyle='-', linewidth=2.0,color='red')
    plt.show();
    

    fechaSintomas = pd.pivot_table(fechaSintomas, values='SINTOMAS', index=['FECHA_SINTOMAS'], columns=['RESULTADO'], aggfunc=np.sum)

    dias = (fechaSintomas.iloc[-1,:].name-fechaSintomas.iloc[0,:].name).days + 1 + diasAjuste
    fechas_serie = pd.date_range(fechaSintomas.iloc[0,:].name, periods=dias, freq='D')
    fechas_dataFrame = (fechas_serie.to_frame(name='Fecha'))

    fechas = fechas_dataFrame.join(fechaSintomas, lsuffix='si', rsuffix='_other')
    fechas = fechas.set_index('Fecha')

    %timeit fechas['MA'] = fechas.iloc[:,0].rolling(medias_moviles).mean()
    fechas['MA'] = fechas['MA'].replace(np.NaN,0)

    fig = plt.figure()
    ax = fechas[fechas.columns[0:len(fechas.columns) - 1]].plot(kind='bar',title="Sintomas",stacked=True,grid=False, figsize=(15,10))
    ax2 = ax.twinx()
    ax2.plot(fechas[fechas.columns[len(fechas.columns) - 1]].values, linestyle='-', linewidth=2.0,color='red')
    plt.show();


    fechadef = pd.pivot_table(fechadef, values='DEF', index=['FECHA_DEF'], columns=['RESULTADO'], aggfunc=np.sum)

    dias = (fechadef.iloc[-1,:].name-fechadef.iloc[0,:].name).days + 1 + diasAjuste
    fechas_serie = pd.date_range(fechadef.iloc[0,:].name, periods=dias, freq='D')
    fechas_dataFrame = (fechas_serie.to_frame(name='Fecha'))

    fechas = fechas_dataFrame.join(fechadef, lsuffix='si', rsuffix='_other')
    fechas = fechas.set_index('Fecha')

    %timeit fechas['MA'] = fechas.iloc[:,0].rolling(medias_moviles).mean()
    fechas['MA'] = fechas['MA'].replace(np.NaN,0)

    fig = plt.figure()
    ax = fechas[fechas.columns[0:len(fechas.columns) - 1]].plot(kind='bar',title="Defunciones",stacked=True,grid=False, figsize=(15,10))
    ax2 = ax.twinx()
    ax2.plot(fechas[fechas.columns[len(fechas.columns) - 1]].values, linestyle='-', linewidth=2.0,color='red')
    plt.show();


#graficosPorResultadoMA('JC')
graficosPorResultadoMA('JC','RESULTADO in("Positivo SARS-CoV-2")')
#graficosPorResultado('JC', 'TIPO_PACIENTE == "AMBULATORIO"')
#graficosPorResultado('JC', 'RESULTADO in("Positivo SARS-CoV-2","Resultado pendiente","No positivo SARS-CoV-2")')
#graficosPorResultado('JC', 'RESULTADO in("Positivo SARS-CoV-2","No positivo SARS-CoV-2")', 'line')
#AMBULATORIO
#HOSPITALIZADO
#NO ESPECIFICADO